In [57]:
import requests
import json
import os
from dotenv import load_dotenv

In [ ]:
import asyncio
import aiohttp

async def fetch_data(url):
    

In [59]:
load_dotenv()  # take environment variables

True

In [60]:
MTA_API_KEY = os.getenv("MTA_API_KEY")

In [ ]:
# getting data given a route id
async def get_bus_route(route_id):
    url = f"https://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_id}.json?key={MTA_API_KEY}&includePolylines=false&version=2"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()

In [69]:
route_id = "B52" # Example route
route_data = await get_bus_route(route_id)
print(route_data)



{'code': 200, 'currentTime': 1746591607343, 'data': {'entry': {'polylines': [], 'routeId': 'MTA NYCT_B52', 'stopGroupings': [{'ordered': True, 'stopGroups': [{'id': '1', 'name': {'name': 'DOWNTOWN BKLYN TILLARY ST via GATES', 'names': ['DOWNTOWN BKLYN TILLARY ST via GATES'], 'type': 'destination'}, 'polylines': [], 'stopIds': ['MTA_504965', 'MTA_504119', 'MTA_304196', 'MTA_304197', 'MTA_304198', 'MTA_304199', 'MTA_304200', 'MTA_304201', 'MTA_308046', 'MTA_304203', 'MTA_304204', 'MTA_304205', 'MTA_304206', 'MTA_304207', 'MTA_306530', 'MTA_304209', 'MTA_304210', 'MTA_304211', 'MTA_304212', 'MTA_304213', 'MTA_304214', 'MTA_307948', 'MTA_307572', 'MTA_307573', 'MTA_307596', 'MTA_307575', 'MTA_307576', 'MTA_307577', 'MTA_302427', 'MTA_302428', 'MTA_302430', 'MTA_302431', 'MTA_307492', 'MTA_302433', 'MTA_302434', 'MTA_901463'], 'subGroups': []}, {'id': '0', 'name': {'name': 'RIDGEWOOD TERM via GATES', 'names': ['RIDGEWOOD TERM via GATES'], 'type': 'destination'}, 'polylines': [], 'stopIds': 

/var/folders/ly/ml1ytmcj1137w_s0w0f7j0g80000gn/T/ipykernel_6200/3558623838.py:2: RuntimeWarning: coroutine 'get_bus_route' was never awaited
  route_data = await get_bus_route(route_id)


In [ ]:
def get_stop_groups(route_data):
    if not route_data['data']['entry']['stopGroupings'][0]['ordered']:
        print(f"{route_data['routeId']} not ordered")
        return
    route_lists = {}
    stopGroups = route_data['data']['entry']['stopGroupings'][0]['stopGroups']
    
    for group in stopGroups:

        route_lists[group['name']['name']] = group['stopIds']
    
    return route_lists

    

In [ ]:
test_route_lists = get_stop_groups(route_data)

In [ ]:
json_path='./graph_data/bus_info_dict.json'
with open(json_path) as json_file:
    bus_info = json.load(json_file)

In [ ]:
print(bus_info['MTA_901601'])

{'name': 'Brooklyn Bridge Park /PIER 6', 'routes': ['B63'], 'direction': 'W', 'link': 'https://bustime.mta.info/m/index?q=901601', 'id': 'MTA_901601', 'latitude': 40.693078, 'longitude': -74.000877, 'osmid': 1567286111}


In [ ]:
json_path='./graph_data/all_routes.json'
with open(json_path) as json_file:
    all_routes = json.load(json_file)

In [ ]:
print(len(all_routes))

357


In [ ]:
def shorten_id(route):
    shortened_id = route['id'].split('_')[1]
    return shortened_id

In [ ]:
shorten_id(all_routes[0])

M34+


In [ ]:
route_data = get_bus_route('B52')
print(route_data)

{'code': 200, 'currentTime': 1746591325581, 'data': {'entry': {'polylines': [], 'routeId': 'MTA NYCT_B52', 'stopGroupings': [{'ordered': True, 'stopGroups': [{'id': '1', 'name': {'name': 'DOWNTOWN BKLYN TILLARY ST via GATES', 'names': ['DOWNTOWN BKLYN TILLARY ST via GATES'], 'type': 'destination'}, 'polylines': [], 'stopIds': ['MTA_504965', 'MTA_504119', 'MTA_304196', 'MTA_304197', 'MTA_304198', 'MTA_304199', 'MTA_304200', 'MTA_304201', 'MTA_308046', 'MTA_304203', 'MTA_304204', 'MTA_304205', 'MTA_304206', 'MTA_304207', 'MTA_306530', 'MTA_304209', 'MTA_304210', 'MTA_304211', 'MTA_304212', 'MTA_304213', 'MTA_304214', 'MTA_307948', 'MTA_307572', 'MTA_307573', 'MTA_307596', 'MTA_307575', 'MTA_307576', 'MTA_307577', 'MTA_302427', 'MTA_302428', 'MTA_302430', 'MTA_302431', 'MTA_307492', 'MTA_302433', 'MTA_302434', 'MTA_901463'], 'subGroups': []}, {'id': '0', 'name': {'name': 'RIDGEWOOD TERM via GATES', 'names': ['RIDGEWOOD TERM via GATES'], 'type': 'destination'}, 'polylines': [], 'stopIds': 

In [ ]:
#getting a dict of all routes 
for route in all_routes[2:100]:
    route_id = shorten_id(route)
    print(route_id)
    route_data = get_bus_route(route_id)
    print(route_id)
    

B52
None
Error: 404
None
Q24
None
Error: 404
None
S66
None
Error: 404
None
Q27
None
Error: 404
None
Q26
None
Error: 404
None
Q28
None
Error: 404
None
B46+
None
Error: 404
None
S61
None
Error: 404
None
SIM15
None
Error: 404
None
S62
None
Error: 404
None
B101
None
Error: 404
None
B44
None
Error: 404
None
B43
None
Error: 404
None
SIM10
None
Error: 404
None
B46
None
Error: 404
None
SIM11
None
Error: 404
None
B45
None
Error: 404
None
B48
None
Error: 404
None
B47
None
Error: 404
None
SIM9
None
Error: 404
None
B49
None
Error: 404
None
SIM8
None
Error: 404
None
SIM7
None
Error: 404
None
SIM6
None
Error: 404
None
B60
None
Error: 404
None
SIM5
None
Error: 404
None
SIM4
None
Error: 404
None
B62
None
Error: 404
None
SIM1C
None
Error: 404
None
SIM3
None
Error: 404
None
B61
None
Error: 404
None
SIM2
None
Error: 404
None
B64
None
Error: 404
None
SIM1
None
Error: 404
None
B63
None
Error: 404
None
S76
None
Error: 404
None
Q36
None
Error: 404
None
S78
None
Error: 404
None
S79+
None
Error: 404
None
BX12+

KeyboardInterrupt: 